In [345]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing, model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
import xml.etree.cElementTree as ET
import gensim
from gensim.test.utils import common_texts

from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import pos_tag
import string
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
import gensim.downloader as api
import spacy

In [346]:
path = './data/restaurants_laptop_train_with_pos.csv'
df = pd.read_csv(path)
punc = []
for p in string.punctuation:
    punc.append(p)
df["text"] = df.apply(lambda r: r["text"].lower(), axis = 1)
df = df[~df.text.isin(punc)]

In [347]:
stop_words = stopwords.words('english')

encoder = preprocessing.LabelEncoder()
df.loc[:, "aspect_tag"] = encoder.fit_transform(df["aspect_tag"])
df.loc[:, "pos"] = encoder.fit_transform(df["pos"])

df = df.drop(df[df.text.isin(stop_words)].index)
num = df.num.unique()
sentences = df.groupby("num")["text"].apply(list).values
aspect_tags = df.groupby("num")["aspect_tag"].apply(list).values
pos_tags = df.groupby("num")["pos"].apply(list).values
polarity_tags = df.groupby("num")["polarity"].apply(list).values

In [348]:
df

,num,text,pos,aspect_tag,polarity
1,s_1,charge,16,1,0
4,s_1,night,7,1,0
6,s_1,skip,16,1,0
7,s_1,taking,16,1,0
9,s_1,cord,7,0,0
...,...,...,...,...,...
63223,s_3432,various,0,1,0
63224,s_3432,vegetables,7,0,0
63227,s_3432,rice,7,0,0
63229,s_3432,glass,7,0,0


In [349]:
new_df = pd.DataFrame(sentences, columns = ["sentences"])
new_df['num'] = num
new_df

,sentences,num
0,"[charge, night, skip, taking, cord, good, batt...",s_1
1,"[however, multi, touch, gestures, large, track...",s_2
2,"[mention, using, months, battery, slowly, less...",s_3
3,"[ , company, sent, whole, new, cord, overnigh...",s_4
4,"['s, even, easy, hook, wireless, networks]",s_5
...,...,...
3427,"[thank, best, buy, putting, computer, together...",s_3428
3428,"[adjust, sensitivity, since, 's, responsive, b...",s_3429
3429,"[couple, years, battery, life, began, diminish...",s_3430
3430,"[system, constantly, overheats, battery, life,...",s_3431


In [350]:
#dataset = api.load("text8")  # load dataset as iterable
#model = Word2Vec(dataset)
#model = Word2Vec(sentences=dataset, vector_size=100, window=5, min_count=2, workers=4)
#model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")

In [351]:
df = df.merge(new_df, on = ['num']).drop(['num'], axis = 1)

In [352]:
df["text"] = df.apply(lambda r: model.wv[r["text"]].mean() if r["text"] in model.wv else 0, axis = 1) 
df["sentences"] = df["sentences"].apply(lambda r:to_vector(r))

In [332]:
def to_vector(sentences):
    score = 0
    for word in sentences: # looping through each word of a single document and spliting through space
        try:
            word_vec =  model.wv[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
            score+=word_vec.mean() # if word is present then append it to temporary dataframe
        except:
            pass


    return score

In [353]:
train, test = train_test_split(df, test_size = 0.2, random_state=42)
le = preprocessing.LabelEncoder()

X_train = train.drop(columns=['polarity'])
X_test = test.drop(columns = ['polarity'])
y_test= test.polarity.values
y_train = train.polarity.values


# le = preprocessing.LabelEncoder()

# X = df.drop(columns=['polarity'])
# # X = encoder.fit_transform(X.to_dict('record'))
# y = df.polarity.values

# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2, random_state=42)
# X_train = X_train.to_dict('record')
# X_test = X_test.to_dict('record')
# encoder = DictVectorizer(sparse=False).fit(X_train) # Only apply fit on the training data
# X_train = encoder.transform(X_train)
# X_test = encoder.transform(X_test)

In [342]:
X_train

,text,pos,aspect_tag,sentences
20182,"[[-1.3429726], [-0.72830087], [0.68553716], [-...",2,1,-0.285092
14934,"[[-0.65416163], [-0.26087844], [0.05443175], [...",16,1,0.136749
19763,"[[-0.16980982], [-0.042012244], [0.29509452], ...",2,1,-0.050700
2442,"[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...",8,0,0.277995
25607,"[[0.041025233], [-0.17981656], [-0.340183], [-...",16,1,-0.075606
...,...,...,...,...
21575,"[[-0.28945324], [0.16655573], [0.16224639], [-...",7,0,-0.195392
5390,"[[-3.0386474], [1.2061127], [-1.8891833], [-1....",7,0,0.026003
860,"[[-0.48449257], [-2.762119], [1.4600674], [-0....",16,1,0.175658
15795,"[[-0.24975982], [-0.35269177], [-0.43002075], ...",16,0,0.084376


In [354]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

C:\Users\sr_te\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [355]:
pred = lr.predict(X_test)
pred, y_test

(array([1, 0, 0, ..., 1, 1, 0], dtype=int64),
 array([-1,  0,  0, ...,  1, -1,  0], dtype=int64))

In [356]:
print('Testing accuracy: {:.2f}%'.format(lr.score(X_test, y_test)*100))

Testing accuracy: 86.93%
